### 모델 훈련
개별 파일(MF.py, KNN.py)에서도 훈련이 가능하지만, 최상위 모듈인 train.py를 사용하여 학습을 진행할 수 있습니다.</br>
train.py 모듈을 터미널에서 실행하는 것으로 학습할 수 있습니다.</br>
train.py와 함께 사용할 수 있는 arguments들은 다음과 같습니다.</br>

**mf 모델 학습용 arguments**
- --mf : Matrix Factorization 모델의 학습 여부를 결정하는 인자입니다. --mf 를 입력하는 경우 학습을 진행하며, --no-mf를 입력하는 경우 해당 모델의 학습을 진행하지 않습니다.(입력하지 않았을 경우에는 학습 진행을 하도록 되어 있습니다.)
- -k : Matrix Factorization 행렬의 크기를 결정하는 인자값입니다. (default = 200)
- -e 또는 --epochs : mf 모델의 학습 횟수를 결정하는 인자입니다. (default = 1)
- -b 또는 --batch_size : 학습 시 사용하는 batch_size를 결정하는 인자입니다.(default = 512)

**knn 모델 학습용 arguments**
- --knn : 최근접 이웃을 활용한 콘텐츠 기반 필터링 모델의 학습 여부를 결정하는 인자입니다. --knn을 입력하는 경우 학습을 진행하며, word2vec 모델과 cbf_data, knn모델을 models 폴더 경로에 생성합니다.(이미 존재하는 경우에는 업데이트) --no-knn을 입력하면 해당 모델의 학습을 진행하지 않습니다.(입력하지 않았을 경우에는 학습 진행을 하도록 되어 있습니다.)
- --vector_size : word2vec 학습에 생성하는 행렬 차원의 수를 결정하는 인자입니다.(default = 100)
- --pretrained : 영화 장르 임베딩을 적용할 때 사용하는 gensim의 사전학습 모델을 결정하는 인자입니다. (default = 'glove-twitter-100')

In [17]:
#CLI
#mf모델만 3 epochs로 학습하라는 명령어(knn 모델은 학습하지 않음)
#두가지 모델을 전무 학습시키고 싶다면 !python trian.py 명령어를 사용하시면 됩니다.
!python train.py
#--no-knn -e 3


1251/1251 [==============================] - 14s 11ms/step - loss: 5.7668 - val_loss: 0.9398
---Saving model---
Save Complete.
---Tokenizing...---
Tokenizing Complete.
---w2v Training...---
(4839, 100)
w2v Training Complete.
---pre-trained w2v loading...---
loading Complete.
---knn Training...---
---knn Done.---


### 인기도 기반 추천 시스템 테스트
베이스라인 모델의 인기도 기반 추천은 사용자들이 가장 많이 평점을 남긴 영화 순으로 추천하도록 설계되어 있습니다.</br>
관련 코드는 models 폴더의 impersonal.py 코드에 작성되어 있습니다.

In [18]:
from models.impersonal import popular
popular(10)

[2858, 260, 1196, 1210, 480, 2028, 589, 2571, 1270, 593]

### 협업 필터링 모델 테스트
Matrix Factorization 기반 모델 코드의 경우 train.py를 통해 학습한 mf.h5 파일을 로드하여 예측을 진행합니다.

In [19]:
from models.MF import MF

mf = MF('./models/mf.h5')

In [20]:
userid = 1
mf.predict(userid)

[858, 527, 1148, 908, 260, 1198, 318, 930, 50, 2762]

### 콘텐츠 기반 필터링 모델 테스트
최근접 이웃 알고리즘을 활용한 콘텐츠 기반 필터링 모델의 경우 train.py를 통해 학습한 knn.joblib 파일을 로드하여 예측을 진행합니다.

In [21]:
from models.KNN import KNN

knn = KNN('models/knn.joblib')

In [23]:
USER_ID = 1
TOP_NUM=10
knn.predict(USER_ID, TOP_NUM)

array([2081, 2080, 2138,  239, 1566,  919, 2102,  588, 2078,   48],
      dtype=int64)

### 하이브리드 모델 테스트
최근접 이웃 알고리즘과 협업필터링을 병합한 모델을 통해 학습하고 예측을 진행<BR>
독립된 추천 결과를 조합 


In [1]:
from models.hybrid import Hybrid_1, Hybrid_2
import numpy as np
from models.MF import MF
from models.KNN import KNN

In [2]:
# 각 모델 생성
mf = MF('./models/mf.h5')
knn = KNN('models/knn.joblib')
alpha = 0.5  # 가중치 조절 파라미터(KNN : alpha , MF : 1-alpha)
hybrid_c = Hybridcombine(knn, mf, alpha)
USER_ID = 1
TOP_NUM = 10
hybrid_c.predict(USER_ID, TOP_NUM)

([2081, 858, 2080, 527, 2138, 1148, 239, 908, 1566, 260],
 array([2081, 2080, 2138,  239, 1566,  919, 2102,  588, 2078,   48],
       dtype=int64),
 [858, 527, 1148, 908, 260, 1198, 318, 930, 50, 2762])

#### 콘텐츠 기반 모델로 출력한 user에 대한 영화 top 10

In [31]:
from utils.Dataloader import load_ratings, load_movies

DATA_PATH = "./datasets/"
movies_df = load_movies(DATA_PATH)

predicted_values, knn_r, mf_r = hybrid_c.predict(target_user_id, TOP_NUM)
m_titles=[]
for i in predicted_values:
    m_title =movies_df[movies_df['movieId'] ==i]['title'].item()
    m_titles.append(m_title)
m_titles

['Little Mermaid, The (1989)',
 'Godfather, The (1972)',
 'Lady and the Tramp (1955)',
 "Schindler's List (1993)",
 'Watership Down (1978)',
 'Wrong Trousers, The (1993)',
 'Goofy Movie, A (1995)',
 'North by Northwest (1959)',
 'Hercules (1997)',
 'Star Wars: Episode IV - A New Hope (1977)']

### 콘텐츠 기반 정보를 협업 필터링에 적용
사용자의 평가점수가 아닌 콘텐츠기반 사용자 프로파일을 이용

In [2]:
# 필요 라이브러리
import numpy as np
from models.KNN import KNN
from models.MF import MF
import joblib
from utils.Dataloader import load_ratings, load_movies, load_users

# 모델 로드 및 초기화
knn_model = KNN('models/knn.joblib')
mf_model = MF('./models/mf.h5')

# 사용자, 영화, 평점 데이터 로딩 (users_df, movies_df, ratings_df)
DATA_PATH = "./datasets/"
users_df = load_users(DATA_PATH)
movies_df = load_movies(DATA_PATH)
ratings_df = load_ratings(DATA_PATH)

# Hybrid 모델 생성
hybrid_model = Hybrid_2(knn_model, mf_model)

# 데이터 로딩 및 처리
hybrid_model.load_data(users_df, movies_df, ratings_df)

# 특정 사용자 ID 설정
target_user_id = 1004

# 특정 사용자의 추천 목록 출력
top_num = 10
recommended_movies = hybrid_model.recommend(target_user_id, top_num)
print(f"User {target_user_id} Recommendations (Hybrid):", recommended_movies)


KeyboardInterrupt: 

### Surprise 활용 recommender

In [ ]:
from models.Surprise_hybrid import SHybridRecommender
from utils.Dataloader import load_ratings, load_movies, load_users

DATA_PATH = "./datasets/"
users_df = load_users(DATA_PATH)
movies_df = load_movies(DATA_PATH)
ratings_df = load_ratings(DATA_PATH)

hybrid_recommender = SHybridRecommender(users_df, movies_df, ratings_df)

user_id = 1004
recommended_movie_indices = hybrid_recommender.hybrid_recommend(user_id, num_recommendations=10)
print("Recommended Movies:")
for idx, movie_idx in enumerate(recommended_movie_indices, start=1):
    movie_title = movies_df.iloc[movie_idx]['movie_title']
    print(f"{idx}. {movie_title}")

In [7]:
from models.Surprise_hybrid import EnsembleRecommender
import numpy as np
from utils.Dataloader import load_ratings, load_movies, load_users
from scipy.sparse import csr_matrix

DATA_PATH = "./datasets/"
rating_df = load_ratings(DATA_PATH)
movie_df = load_movies(DATA_PATH)

user_ids = rating_df["userId"].unique().tolist() 
num_all_user = len(user_ids)

# randomly select 20% users from rating dataset 
np.random.seed(123)
rand_userid = np.random.choice(user_ids, size=int(num_all_user * 0.1), replace=False)
sample_df = rating_df.loc[rating_df['userId'].isin(rand_userid)]

# Create user-item rating matrix
def movie_use_matrix_pivot(df_):
    mu_matrix = df_.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    # compress original matrix
    mu_matrix_cp = csr_matrix(mu_matrix.values)
    return mu_matrix, mu_matrix_cp

rating_matrix, rating_matrix_cp = movie_use_matrix_pivot(sample_df)

# Create some dummy item vectors for illustration
num_movies = len(sample_df['movieId'].unique())
num_features = 10
item_vector = np.random.rand(num_movies, num_features)

# Initialize the EnsembleRecommender
Ensemble = EnsembleRecommender(sample_df, movie_df, rating_matrix_cp, item_vector)

# user 1004 has more than 150 ratings
Ensemble.Recommend(1004)


,title,genres
movieId,,


### 평가지표 NDCG(수정 필요)

In [32]:
from utils.Dataloader import load_ratings

# 데이터 폴더 경로
DATA_PATH = "./datasets/"

# ratings 데이터 로드
ratings_df = load_ratings(DATA_PATH)

# 특정 user_id
target_user_id = 1

# 특정 user_id의 평점 데이터
target_user_ratings = ratings_df[ratings_df['userId'] == target_user_id]

# 평점을 기준으로 내림차순 정렬하여 top 10개 출력
top_rated_movies = target_user_ratings.sort_values(by='rating', ascending=False).head(10)

print(f"User {target_user_id}의 Top 10 평점 영화:")
for rank, row in top_rated_movies.iterrows():
    movie_id = row['movieId']
    rating = row['rating']
    print(f"{rank+1}. 영화 ID: {movie_id}, 평점: {rating}")


User 1의 Top 10 평점 영화:
1. 영화 ID: 1193, 평점: 5
47. 영화 ID: 1029, 평점: 5
41. 영화 ID: 1, 평점: 5
19. 영화 ID: 3105, 평점: 5
42. 영화 ID: 1961, 평점: 5
24. 영화 ID: 527, 평점: 5
38. 영화 ID: 1022, 평점: 5
15. 영화 ID: 1035, 평점: 5
26. 영화 ID: 48, 평점: 5
46. 영화 ID: 1028, 평점: 5


In [33]:
# 테스트 데이터 로드
ratings_df = load_ratings('./datasets/')

# 테스트를 위한 특정 user_id
target_user_id = 1004

# top N 영화 예측
TOP_NUM = 10

'''predicted_values, knn_r, mf_r = hybrid_c.predict(target_user_id, TOP_NUM)
hybrid_predictions = predicted_values'''

# 실제 평가 데이터 가져오기- 수정 필요
actual_ratings = ratings_df[ratings_df['userId'] == target_user_id]
actual_ratings = actual_ratings.sort_values(by='rating', ascending=False)

# NDCG 계산
def ndcg(actual, predicted, k):
    idcg = sum(1 / (np.log2(rank + 2)) for rank in range(min(k, len(actual))))
    dcg = sum(1 / (np.log2(rank + 2)) if movie_id in predicted else 0 for rank, (movie_id, _) in enumerate(actual.items()))
    return dcg / idcg

# NDCG 계산 및 출력
print(f"User {target_user_id}의 NDCG@{TOP_NUM}: {ndcg(actual_ratings['movieId'], hybrid_predictions, TOP_NUM):.4f}")


User 1004의 NDCG@10: 0.0000


In [16]:
# NDCG 계산
def ndcg(actual, predicted, k):
    idcg = sum(1 / (np.log2(rank + 2)) for rank in range(min(k, len(actual))))
    dcg = sum(1 / (np.log2(rank + 2)) if movie_id in predicted else 0 for rank, (movie_id, _) in enumerate(actual.items()))
    return dcg / idcg

# NDCG 계산 및 출력
print(f"User {target_user_id}의 NDCG@{TOP_NUM}: {ndcg(actual_ratings['movieId'], hybrid_predictions, TOP_NUM):.4f}")
print(f"User {target_user_id}의 NDCG@{TOP_NUM}: {ndcg(actual_ratings['movieId'], knn_r, TOP_NUM):.4f}")
print(f"User {target_user_id}의 NDCG@{TOP_NUM}: {ndcg(actual_ratings['movieId'], mf_r, TOP_NUM):.4f}")


User 1004의 NDCG@10: 0.0000
User 1004의 NDCG@10: 0.0000
User 1004의 NDCG@10: 0.0000


In [10]:
# NDCG 계산
def ndcg(actual, predicted, k):
    idcg = sum(1 / (np.log2(rank + 2)) for rank in range(min(k, len(actual))))
    dcg = sum(1 / (np.log2(rank + 2)) if movie_id in predicted else 0 for rank, (movie_id, _) in enumerate(actual.iterrows()))
    return dcg / idcg

In [13]:
# NDCG 계산 및 출력
print(f"User {target_user_id}의 NDCG@{TOP_NUM}: {ndcg(actual_ratings.index, hybrid_predictions, TOP_NUM):.4f}")

AttributeError: 'Index' object has no attribute 'iterrows'

In [160]:
knn_r = knn.predict(1004, 10)
mf_r = mf.predict(1004, 10)
print(knn_r)
print(mf_r)

[1205 1215  258   70 2054 1210 2322  673 1264 1196]
[1198, 260, 1036, 1200, 1196, 858, 318, 1240, 589, 1214]


In [161]:
knn_rank = {idx: 1 / (i + 1) for i, idx in enumerate(knn_r)}
mf_rank = {idx: 1 / (i + 1) for i, idx in enumerate(mf_r)}
print(knn_rank)
print(mf_rank)

{1205: 1.0, 1215: 0.5, 258: 0.3333333333333333, 70: 0.25, 2054: 0.2, 1210: 0.16666666666666666, 2322: 0.14285714285714285, 673: 0.125, 1264: 0.1111111111111111, 1196: 0.1}
{1198: 1.0, 260: 0.5, 1036: 0.3333333333333333, 1200: 0.25, 1196: 0.2, 858: 0.16666666666666666, 318: 0.14285714285714285, 1240: 0.125, 589: 0.1111111111111111, 1214: 0.1}


In [162]:
alpha = 0.5
combined_scores = {}
for idx, knn_rank in knn_rank.items():
    combined_scores[idx] = combined_scores.get(idx, 0) + (alpha * knn_rank)
for idx, mf_rank in mf_rank.items():
    combined_scores[idx] = combined_scores.get(idx, 0) + ((1 - alpha) * mf_rank)

print(combined_scores)

{1205: 0.5, 1215: 0.25, 258: 0.16666666666666666, 70: 0.125, 2054: 0.1, 1210: 0.08333333333333333, 2322: 0.07142857142857142, 673: 0.0625, 1264: 0.05555555555555555, 1196: 0.15000000000000002, 1198: 0.5, 260: 0.25, 1036: 0.16666666666666666, 1200: 0.125, 858: 0.08333333333333333, 318: 0.07142857142857142, 1240: 0.0625, 589: 0.05555555555555555, 1214: 0.05}


In [163]:
sorted_combined = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
print(sorted_combined)

[(1205, 0.5), (1198, 0.5), (1215, 0.25), (260, 0.25), (258, 0.16666666666666666), (1036, 0.16666666666666666), (1196, 0.15000000000000002), (70, 0.125), (1200, 0.125), (2054, 0.1), (1210, 0.08333333333333333), (858, 0.08333333333333333), (2322, 0.07142857142857142), (318, 0.07142857142857142), (673, 0.0625), (1240, 0.0625), (1264, 0.05555555555555555), (589, 0.05555555555555555), (1214, 0.05)]


In [164]:
top_10_indices = [idx for idx, _ in sorted_combined[:10]]
print(top_10_indices)

[1205, 1198, 1215, 260, 258, 1036, 1196, 70, 1200, 2054]
